In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # for Yelp

# import json
# import pandas as pd

# input_file = "/content/drive/MyDrive/FakeReviewDetectionProject/Datasets/yelp_academic_dataset_review.json"
# output_file = "/content/drive/MyDrive/FakeReviewDetectionProject/Datasets/Yelp.csv"

# # Open the JSON file in streaming mode
# with open(input_file, "r", encoding="utf-8") as file:
#     data_lines = []

#     for i, line in enumerate(file):  # Read line-by-line to avoid memory overload
#         if i >= 20000:  # Stop after 20k entries
#             break
#         data_lines.append(json.loads(line.strip()))  # Load each JSON object

# # Convert to DataFrame
# df = pd.DataFrame(data_lines)

# # Save as CSV
# df.to_csv(output_file, index=False)

# print("CSV file saved with first 20k entries.")

In [27]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/FakeReviewDetectionProject/Datasets/Amazon1.csv")

df.head()

,Unnamed: 0,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [28]:
df = df[['reviewText', 'overall']]

In [29]:
df.head()

,reviewText,overall
0,They look good and stick good! I just don't li...,4
1,These stickers work like the review says they ...,5
2,These are awesome and make my phone look so st...,5
3,Item arrived in great time and was in perfect ...,4
4,"awesome! stays on, and looks great. can be use...",5


In [30]:
df.rename(columns={'reviewText': 'Review'}, inplace=True)
df.rename(columns={'overall': 'Rating'}, inplace=True)

<ipython-input-30-20523d800882>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'reviewText': 'Review'}, inplace=True)
<ipython-input-30-20523d800882>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'overall': 'Rating'}, inplace=True)


In [31]:
import numpy as np
import string
from nltk.tokenize import sent_tokenize


import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Remove special characters – Keep only alphanumeric text.**


**Remove extra spaces – Normalize whitespace.**

**Remove numbers – Numbers usually don't contribute to review meaning.**

**Remove stopwords – Common words like "the", "is" don’t add value.**

**Lemmatize words – Convert words to their base form (e.g., "running" → "run").**

In [32]:
df.head()

,Review,Rating
0,They look good and stick good! I just don't li...,4
1,These stickers work like the review says they ...,5
2,These are awesome and make my phone look so st...,5
3,Item arrived in great time and was in perfect ...,4
4,"awesome! stays on, and looks great. can be use...",5


In [33]:
df = df[:20000]

In [34]:
df = df.dropna(subset=['Review'])
df = df.dropna(subset=['Rating'])

In [35]:
# if no ratings column, will use this

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Step 1: Convert reviews to numerical features
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust features as needed
X = vectorizer.fit_transform(df['Review'])

# Step 2: Apply KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(X)

# Step 3: Map clusters to ratings (1 to 5)
cluster_mapping = {c: i+1 for i, c in enumerate(sorted(set(df['Cluster']), key=lambda x: X[df['Cluster'] == x].sum()))}
df['Rating'] = df['Cluster'].map(cluster_mapping)

# Drop the temporary Cluster column
df.drop(columns=['Cluster'], inplace=True)

print(df[['Review', 'Rating']].head())

                                              Review  Rating
0  They look good and stick good! I just don't li...       3
1  These stickers work like the review says they ...       1
2  These are awesome and make my phone look so st...       1
3  Item arrived in great time and was in perfect ...       1
4  awesome! stays on, and looks great. can be use...       4


In [36]:
def clean_review(text):
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = text.split()
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply cleaning
df['Cleaned_Review'] = df['Review'].apply(clean_review)

**Extracting main features**

In [37]:
# Compute product average ratings
avg_rating = df['Rating'].mean()

# Function to extract features
def extract_features(text, rating):
    words = text.split()
    unique_words = set(words)
    sentences = sent_tokenize(text)

    review_length = len(words)
    sentence_count = len(sentences)
    word_diversity = len(unique_words) / review_length if review_length > 0 else 0
    stopword_ratio = sum(1 for word in words if word in stopwords.words('english')) / review_length if review_length > 0 else 0
    punctuation_count = sum(1 for char in text if char in string.punctuation)
    extreme_rating = 1 if rating in [1, 5] else 0
    deviation_from_avg = abs(rating - avg_rating)

    return [review_length, sentence_count, word_diversity, stopword_ratio, punctuation_count, extreme_rating, deviation_from_avg]

# Apply feature extraction
df[['Review_Length', 'Sentence_Count', 'Word_Diversity', 'Stopword_Ratio',
    'Punctuation_Count', 'Extreme_Rating', 'Deviation_From_Avg']] = df.apply(
    lambda row: extract_features(row['Cleaned_Review'], row['Rating']), axis=1, result_type='expand'
)

In [38]:
df.head(2)

,Review,Rating,Cleaned_Review,Review_Length,Sentence_Count,Word_Diversity,Stopword_Ratio,Punctuation_Count,Extreme_Rating,Deviation_From_Avg
0,They look good and stick good! I just don't li...,3,hey look good stick good dont like rounded sha...,19.0,1.0,0.894737,0.0,0.0,0.0,0.469155
1,These stickers work like the review says they ...,1,hese sticker work like review say hey stick gr...,16.0,1.0,0.937500,0.0,0.0,1.0,2.469155


**Tokenizing using word2vec**

In [39]:
import gensim
import numpy as np

# Tokenize reviews into lists of words
df['Tokenized_Review'] = df['Cleaned_Review'].apply(lambda x: x.split())

# Train Word2Vec model
word2vec_model = gensim.models.Word2Vec(sentences=df['Tokenized_Review'], vector_size=700, window=5, min_count=2, workers=4)

# Function to get review embeddings (average of word vectors)
def get_review_embedding(words):
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(700)

# Apply embedding extraction
df['Embedding'] = df['Tokenized_Review'].apply(get_review_embedding)

In [40]:
df.head(2)

,Review,Rating,Cleaned_Review,Review_Length,Sentence_Count,Word_Diversity,Stopword_Ratio,Punctuation_Count,Extreme_Rating,Deviation_From_Avg,Tokenized_Review,Embedding
0,They look good and stick good! I just don't li...,3,hey look good stick good dont like rounded sha...,19.0,1.0,0.894737,0.0,0.0,0.0,0.469155,"[hey, look, good, stick, good, dont, like, rou...","[0.021237949, 0.08787922, -0.3021761, 2.997799..."
1,These stickers work like the review says they ...,1,hese sticker work like review say hey stick gr...,16.0,1.0,0.937500,0.0,0.0,1.0,2.469155,"[hese, sticker, work, like, review, say, hey, ...","[0.08099937, 0.081815995, -0.29075378, 0.02548..."


In [41]:
# if alr has a deceptive label

# df['Fake_Label'] = df['deceptive'].apply(lambda x: 0 if x == 'truthful' else 1)


In [42]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from sklearn.preprocessing import StandardScaler

In [43]:
# Load dataset (Assume df already contains extracted features)
X = df[['Review_Length', 'Sentence_Count', 'Word_Diversity', 'Stopword_Ratio',
        'Punctuation_Count', 'Extreme_Rating', 'Deviation_From_Avg']].values

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define autoencoder model
autoencoder = Sequential([
    Dense(64, activation="relu", input_shape=(X_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.2),

    Dense(32, activation="relu"),
    Dense(16, activation="relu"),  # Bottleneck
    Dense(32, activation="relu"),
    Dropout(0.2),
    BatchNormalization(),

    Dense(64, activation="relu"),
    Dense(X_scaled.shape[1], activation="tanh")  # Reconstruct input
])

# Compile model
autoencoder.compile(optimizer="adam", loss="mse")

# Train autoencoder on assumed real reviews (~70%)
n_train = int(0.7 * len(X_scaled))
autoencoder.fit(X_scaled[:n_train], X_scaled[:n_train], epochs=50, batch_size=32, validation_split=0.1)

# Compute reconstruction error
reconstructions = autoencoder.predict(X_scaled)
errors = np.mean(np.abs(X_scaled - reconstructions), axis=1)

# Set threshold (Top 30% highest errors as fake)
thresh = np.percentile(errors, 70)
df['Fake_Label'] = (errors > thresh).astype(int)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


394/394 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.5965 - val_loss: 0.4227
Epoch 2/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3904 - val_loss: 0.4029
Epoch 3/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3840 - val_loss: 0.3974
Epoch 4/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3952 - val_loss: 0.3964
Epoch 5/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3621 - val_loss: 0.3941
Epoch 6/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.4033 - val_loss: 0.3932
Epoch 7/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3532 - val_loss: 0.3917
Epoch 8/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3325 - val_loss: 0.3912
Epoch 9/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3591 - val_loss: 0.3914
Epoch 10/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3693 - val_loss: 0.3903
Epoch 11/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3928 - val_loss: 0.3897
Epoch 12/50
394/394 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.

In [44]:
df.head(2)

,Review,Rating,Cleaned_Review,Review_Length,Sentence_Count,Word_Diversity,Stopword_Ratio,Punctuation_Count,Extreme_Rating,Deviation_From_Avg,Tokenized_Review,Embedding,Fake_Label
0,They look good and stick good! I just don't li...,3,hey look good stick good dont like rounded sha...,19.0,1.0,0.894737,0.0,0.0,0.0,0.469155,"[hey, look, good, stick, good, dont, like, rou...","[0.021237949, 0.08787922, -0.3021761, 2.997799...",0
1,These stickers work like the review says they ...,1,hese sticker work like review say hey stick gr...,16.0,1.0,0.937500,0.0,0.0,1.0,2.469155,"[hese, sticker, work, like, review, say, hey, ...","[0.08099937, 0.081815995, -0.29075378, 0.02548...",1


In [45]:
print(df['Fake_Label'].sum()/df.shape[0])

0.29989493170560866


In [46]:
df.to_csv("/content/drive/MyDrive/FakeReviewDetectionProject/Dataset_Labelled_Extracted/Amazon_700f.csv", index=False)

In [47]:
df.shape

(19987, 13)